In [ ]:
# Josue Castellanos (2024)
# This script is for interferogram analysis as part of the PX Beamlines LDRD Project
import os
from glob import glob
import os
from PIL import Image
import pandas as pd
from io import StringIO


# carpenmentalize the process to work with a certain data. 

# put images together as a motor coordinate, could be negative

# 3 or 4th data file in txt have coordinates

# Get coordinates from txt, associate the coordinates to KCube DC 2 Goal, Actual (Actual is the best alignment)

# Rotate - left side is west , right side is east, 90 degree rotation -90 or +90

# X motor is negative

# what direction is it scanning?

# radius coparison 

# Stitch - actual distance and camera, effective pixel size , subtract one image from another to see the difference

# Where are we on the surface, scan 601 , lets pass into the code the cropped rotated image that has the coordinate on the file name so we can reference that to where it is in space


In [ ]:
def GlobFileNames(dirNum):
    basepath = '/Volumes/SanDisk/Image Single Motor Scan'  # My sandisk drive
    #basepath = '/Beamline Controls/BCS Setup Data/240625'
    
    dir_path = f'{basepath} 000{dirNum} Images/*.Png'
    
    # if not os.path.isfile(dir_path):
    #     print(f"File not found: {dir_path}")
    #     return
    
    file_list = glob(dir_path)  # Sort list
    file_list.sort()
    
    return file_list

In [ ]:
def ExtractData(fp):
    # Read the file content
    with open(fp, 'r') as file:
        lines = file.readlines()

    # Find the line where DATA starts
    for i, line in enumerate(lines):
        if 'DATA' in line:
            data_start_idx = i + 1
            break
    
    return data_start_idx, lines

In [ ]:
def ReadData2DF(start_index, lines):
    # Read the data into a pandas DataFrame
    data_lines = lines[start_index:]
    data_str = ''.join(data_lines)
    
    # Convert the data into a DataFrame
    data = StringIO(data_str)
    df = pd.read_csv(data, sep='\t')

    return df

In [ ]:
def ModifyFilename(file):
    basename = os.path.basename(file)
    # Remove a specific part of the string
    modified_basename = basename.replace('Image Single Motor Scan ', '')
    modified_basename = modified_basename.replace(' ', '_')

    return modified_basename

In [ ]:
def CropRotateImagesToRegion(file, startXpixel, endXpixel, startYpixel, endYpixel, coordinate):
    # Open an image file
    with Image.open(file) as img:
        # Crop the image
        cropped_img = img.crop((startXpixel, startYpixel, endXpixel, endYpixel))
        # Rotate the image 90 degrees, Pillow 90 default is counterclockwise so -90 is needed for clockwise
        rotated_img = cropped_img.rotate(-90, expand=True)
        # Modify the new file name of image
        mod_name = ModifyFilename(file)
        # Save the cropped image
        cropped_file = f"Cropped_Rotated_{coordinate}mm_" + mod_name
        rotated_img.save(cropped_file)
        print(f"Cropped image saved as: {cropped_file}")
        
        return cropped_file

In [ ]:
def CropImages(files, startXpixel, endXpixel, startYpixel, endYpixel, coords):
    croppedFiles = []
    for file, coordinate in zip(files, coords):
        cropped_file = CropRotateImagesToRegion(file, startXpixel, endXpixel, startYpixel, endYpixel, coordinate)
        if cropped_file:
            croppedFiles.append(cropped_file)

    return croppedFiles

In [ ]:
def GetTwoImages(coordinate1, point1, coordinate2, point2):
    return

In [ ]:
def ReadModifiedImages(dirNum):
    # Read the cropped images from disk and return them as a list
    cropped_files = [os.path.join(dirNum, f) for f in os.listdir(dirNum) if "_cropped" in f]
    return cropped_files

In [ ]:
# Below is testing the functionality
# Replace with the desired directory number 000599, 000600, 000601...etc
file_index = 599
file_names = GlobFileNames(file_index)

for f in file_names:
    print(f)

In [ ]:
# Replace with the actual path to the text file
header_path = (f'/Volumes/SanDisk/Image Single Motor Scan 000{file_index}.txt' ) 

# Extract All lines in txt file and provide a start index 
start_index, lines = ExtractData(header_path)
data = ReadData2DF(start_index, lines)

In [ ]:
def CleanData(dict, column):
    df = dict.iloc[:,column - 1].tolist()
    return df

column = 4    # Column in dataframe
coordinates = CleanData(data, column)
coordinates

In [ ]:
startXpixel = 780
endXpixel = 2500
startYpixel = 10
endYpixel = 1400

cropped_files = CropImages(file_names, startXpixel, endXpixel, startYpixel, endYpixel, coordinates)


**TESTING BELOW**